# MNIST Training with ResNet-style MLP

In [ ]:
import sys, os
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath("")), ""))

import needle as ndl
import needle.nn as nn
import numpy as np

np.random.seed(0)


def ResidualBlock(dim, hidden_dim, norm=nn.BatchNorm1d, drop_prob=0.1):
    return nn.Sequential(
            nn.Residual(
                fn=nn.Sequential(
                        nn.Linear(dim, hidden_dim),
                        norm(hidden_dim),
                        nn.ReLU(),
                        nn.Dropout(drop_prob),
                        nn.Linear(hidden_dim, dim),
                        norm(dim),
                )
            ),
            nn.ReLU()
    )


def MLPResNet(
    dim,
    hidden_dim=100,
    num_blocks=3,
    num_classes=10,
    norm=nn.BatchNorm1d,
    drop_prob=0.1,
):
    return nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.ReLU(),
            nn.Sequential(
                    *[ResidualBlock(
                        hidden_dim, hidden_dim // 2, norm=norm, drop_prob=drop_prob
                    ) for _ in range(num_blocks)]
            ),
            nn.Linear(hidden_dim, num_classes),
    )


def epoch(dataloader, model, opt=None):
    np.random.seed(4)
    loss_fn = nn.SoftmaxLoss()
    hit = 0
    loss_cum = 0

    if opt:
        model.train()
        for idx, data in enumerate(dataloader):
            x, y = data
            y_pred = model(x)
            opt.reset_grad()
            loss = loss_fn(y_pred, y)
            loss_cum += loss.numpy()
            hit += (y_pred.numpy().argmax(1) == y.numpy()).sum()
            loss.backward()
            opt.step()
    else:
        model.eval()
        for idx, data in enumerate(dataloader):
            x, y = data
            y_pred = model(x)
            hit += (y_pred.numpy().argmax(1) == y.numpy()).sum()
            loss = loss_fn(y_pred, y)
            loss_cum += loss.numpy()

    accuracy = hit / len(dataloader.dataset)
    error = 1 - accuracy
    avg_loss = loss_cum / dataloader.batch_size

    return error, avg_loss


def train_mnist(
    batch_size=100,
    epochs=10,
    optimizer=ndl.optim.Adam,
    lr=0.001,
    weight_decay=0.001,
    hidden_dim=100,
    data_dir="data",
):
    np.random.seed(4)
    tr_im_path = os.path.join(data_dir, "train-images-idx3-ubyte.gz")
    tr_lb_path = os.path.join(data_dir, "train-labels-idx1-ubyte.gz")
    te_im_path = os.path.join(data_dir, "t10k-images-idx3-ubyte.gz")
    te_lb_path = os.path.join(data_dir, "t10k-labels-idx1-ubyte.gz")
    tr_dataset = ndl.data.MNISTDataset(tr_im_path, tr_lb_path)
    te_dataset = ndl.data.MNISTDataset(te_im_path, te_lb_path)

    tr_dataloader = ndl.data.DataLoader(tr_dataset, batch_size=batch_size, shuffle=True)
    te_dataloader = ndl.data.DataLoader(te_dataset)
    model = MLPResNet(784, hidden_dim=hidden_dim)
    opt = optimizer(model.parameters(), lr=lr, weight_decay=weight_decay)

    for e in range(epochs):
        print(f"Epoch: {e}")
        tr_acc, tr_loss = epoch(tr_dataloader, model, opt)
    te_acc, te_loss = epoch(te_dataloader, model)
    return (tr_acc, tr_loss, te_acc, te_loss)


train_mnist(data_dir="../needle/data/data", epochs=10)

Epoch: 0
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
